In [ ]:
import sys

rootpath = '/home/student/ROI/Spark/'
datapath = f'{rootpath}datasets/'
sys.path.append(rootpath)
import pyspark_helpers as pyh
from pyspark_helpers import *
sc, spark, conf = initspark()

import pandas as pd
import matplotlib as mp
import numpy
from matplotlib import pyplot as plt

from pyspark_helpers import display


### The following helper function shows the building of stages to convert categorical and numeric columns into Vectorized versions using a Pipeline instead of building the steps as a series of DataFrames

In [ ]:
def MakeMLDataFramePipeline(df, categorical_features, numeric_features, target_label = None, target_is_categorical = True):
    from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler, StringIndexerModel
    from pyspark.ml import Pipeline

    stages = []

    for c in categorical_features:
        stringIndexer = StringIndexer(inputCol = c, outputCol = c + '_Index')
        encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[c + "_classVec"])
        stages += [stringIndexer, encoder]
        
    if target_is_categorical:
        label_stringIdx = StringIndexer(inputCol = target_label, outputCol = 'label')
        stages += [label_stringIdx]

    assemblerInputs = [c + "_classVec" for c in categorical_features] + numeric_features
    assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
    stages += [assembler]

    pipeline = Pipeline(stages = stages)

    dfML = pipeline.fit(df).transform(df)
    #dfx = dfx.select(['label', 'features'] + cols)
    #catindexes = {x.getOutputCol() : x.labels for x in dfML.stages if isinstance(x, StringIndexerModel)}
    return dfx, catindexes



### Read the same set of data as the previous chapter

In [ ]:
filename = 'bank.csv'
df = spark.read.csv(f'{datapath}/finance/{filename}', header = True, inferSchema = True)
display(df)

# Save a pointer to the raw data
dfRawFile = df



### Use the same categorical and numeric features

In [ ]:
# Let's just keep a few fields to start with for simplicity
numeric_features = ['age','balance', 'duration', 'pdays']
categorical_features = ['job', 'marital', 'education', 'housing', 'loan', 'contact', 'campaign', 'poutcome', 'deposit']

# numeric_features = ['balance', 'duration', 'age']
# categorical_features = ['marital', 'education']
target_label = 'default'


df = dfRawFile.select(numeric_features + categorical_features + [target_label])
display(df)
print(df.take(1))

### Try this using the original helper vs the Pipeline version to see if there is a time difference

In [ ]:
from timeit import default_timer as timer

start = timer()

#dfML = pyh.MakeMLDataFramePipeline(df, categorical_features, numeric_features, target_label)
dfML = pyh.MakeMLDataFrame(df, categorical_features, numeric_features, target_label)

display(dfML)
dfML.printSchema()
labelCnt = dfML.groupBy('label').count()
display(labelCnt)

end = timer()
print('time to run', end - start)


### Train and test as normal

In [ ]:
train, test = dfML.randomSplit([.7,.3], seed = 1000)
print (f'Training set row count {train.count()}')
print (f'Testing set row count {test.count()}')
      

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
dtModel = dt.fit(train)
print('DT Trained')


In [ ]:
dtPredictions, dtLog = pyh.predict_and_evaluate(dtModel, test)


### Pipelines and writing your own models

In [ ]:
from pyspark.ml.pipeline import Estimator, Model, Pipeline, Transformer
from pyspark.ml.param.shared import *
from pyspark.sql.functions import avg, stddev_samp


class myTransformer(Transformer):
    pass

class HasMean(Params):

    mean = Param(Params._dummy(), "mean", "mean", 
        typeConverter=TypeConverters.toFloat)

    def __init__(self):
        super(HasMean, self).__init__()

    def setMean(self, value):
        return self._set(mean=value)

    def getMean(self):
        return self.getOrDefault(self.mean)
class HasStandardDeviation(Params):

    stddev = Param(Params._dummy(), "stddev", "stddev", 
        typeConverter=TypeConverters.toFloat)

    def __init__(self):
        super(HasStandardDeviation, self).__init__()

    def setStddev(self, value):
        return self._set(stddev=value)

    def getStddev(self):
        return self.getOrDefault(self.stddev)

class HasCenteredThreshold(Params):

    centered_threshold = Param(Params._dummy(),
            "centered_threshold", "centered_threshold",
            typeConverter=TypeConverters.toFloat)

    def __init__(self):
        super(HasCenteredThreshold, self).__init__()

    def setCenteredThreshold(self, value):
        return self._set(centered_threshold=value)

    def getCenteredThreshold(self):
        return self.getOrDefault(self.centered_threshold)
    
class NormalDeviation(Estimator, HasInputCol, 
        HasPredictionCol, HasCenteredThreshold):

    def _fit(self, dataset):
        c = self.getInputCol()
        mu, sigma = dataset.agg(avg(c), stddev_samp(c)).first()
        return (NormalDeviationModel()
            .setInputCol(c)
            .setMean(mu)
            .setStddev(sigma)
            .setCenteredThreshold(self.getCenteredThreshold())
            .setPredictionCol(self.getPredictionCol()))

class NormalDeviationModel(Model, HasInputCol, HasPredictionCol,
        HasMean, HasStandardDeviation, HasCenteredThreshold):

    def _transform(self, dataset):
        x = self.getInputCol()
        y = self.getPredictionCol()
        threshold = self.getCenteredThreshold()
        mu = self.getMean()
        sigma = self.getStddev()

        return dataset.withColumn(y, (dataset[x] - mu) > threshold * sigma)

df = sc.parallelize([(1, 2.0), (2, 3.0), (3, 0.0), (4, 99.0)]).toDF(["id", "x"])

normal_deviation = NormalDeviation().setInputCol("x").setCenteredThreshold(1.0)
model  = Pipeline(stages=[normal_deviation]).fit(df)

model.transform(df).show()


### PandasUDF

In [ ]:
df = spark.createDataFrame(pd.DataFrame(pd.Series(range(11)), columns=["x"]))
display(df)

In [ ]:
from pyspark.sql.functions import udf
@udf('int')
def square(x):
      return x * x

display(df.withColumn('x2', square(df.x)))


In [ ]:
#! pip install pyarrow
#import pyarrow
from pyspark.sql.functions import pandas_udf, PandasUDFType
#from pyspark.sql.types import LongType

@pandas_udf('double', PandasUDFType.SCALAR)
def psquare(x):
      return x * x
   
#pandas_square = pandas_udf(psquare, returnType=LongType())

df.withColumn('x3', psquare(df.x)).show()
